In [0]:
-- Create SQL Procedure for ADS_RDS_EVENT_STATUS_ONB_PRODUCTRESULT ETL Process
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_ADS_RDS_EVENT_STATUS_ONB_PRODUCTRESULT(
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
 SQL SECURITY INVOKER
COMMENT 'ETL procedure for ADS_RDS_EVENT_STATUS_ONB_PRODUCTRESULT - converted from notebook format'
AS
BEGIN
    -- Declare local variables
    DECLARE dif_table_name STRING DEFAULT 'gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_STATUS_ONB_PRODUCTRESULT_ADS_MAP_SCD_DIFF';
    DECLARE my_log_id STRING;
    BEGIN
        CALL gap_catalog.log.start_performance_test(
            test_case_name => 'ADS_RDS_PROC',
            step_name => 'ADS_RDS_EVENT_STATUS_ONB_PRODUCTRESULT',
            psource_sys_origin  => 'RDS_ONBOARDINGPRODUCTRESULT',
            process_key => p_process_key,
            target_table => 'EVENT_STATUS',
            log_id => my_log_id,
            p_load_date => p_load_date
        );

        /* Truncate XC Table */
        TRUNCATE TABLE gap_catalog.ads_etl_owner.XC_RDS_EVENT_STATUS_ONB_PRODUCTRESULT;

        /* Fill XC Table */
        INSERT INTO gap_catalog.ads_etl_owner.XC_RDS_EVENT_STATUS_ONB_PRODUCTRESULT 
              SELECT
                     EST_SOURCE_ID
                ,    EST_SOURCE_SYSTEM_ID
                ,    EST_SOURCE_SYS_ORIGIN
                ,    EST_DESC
              FROM (
                    SELECT
                        'PRODUCTRESULT.'||gap_catalog.ads_etl_owner.get_std_code_fn(CDM.value) AS EST_SOURCE_ID
                        ,'RDS' AS EST_SOURCE_SYSTEM_ID
                        ,'RDS_ONBOARDINGPRODUCTRESULT' AS EST_SOURCE_SYS_ORIGIN
                        ,TRIM(BRA.value) AS EST_DESC
                    FROM gap_catalog.ads_etl_owner.DLK_ADS_RDS_ONBOARDINGPRODUCTRESULT CDM
                        JOIN  gap_catalog.ads_etl_owner.DLK_ADS_RDS_ONBOARDINGPRODUCTRESULT BRA
                            ON  BRA.id = CDM.id
                                AND BRA.std_deleted_flag = 'N'
                                AND CAST(from_utc_timestamp(bra.SYS_EFFECTIVE_DATE, 'Europe/Prague') AS DATE) = CAST(p_load_date AS DATE)
                                AND UPPER(TRIM(BRA.sys)) = 'BRASIL'
                                AND UPPER(TRIM(BRA.lang)) = 'CZ'
                                AND to_date('20250914','yyyymmdd') BETWEEN BRA.validfrom AND BRA.validto
                    WHERE CDM.std_deleted_flag = 'N'
                        AND CAST(from_utc_timestamp(cdm.SYS_EFFECTIVE_DATE, 'Europe/Prague') AS DATE) = CAST(p_load_date AS DATE)
                        AND UPPER(TRIM(CDM.sys)) = 'CDM'
                        AND UPPER(TRIM(CDM.lang)) = 'CZ'
                        AND to_date('20250914','yyyymmdd') BETWEEN CDM.validfrom AND CDM.validto 
              )      
        ;

        /* Cleanup DIFF Table */
        DROP TABLE IF EXISTS gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_STATUS_ONB_PRODUCTRESULT_ADS_MAP_SCD_DIFF;

        /* Create DIFF Table */
        create  table gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_STATUS_ONB_PRODUCTRESULT_ADS_MAP_SCD_DIFF
        ( tech_del_flg  char(1),
          tech_new_rec  char(1),
          tech_rid      varchar(255),
          EST_KEY  decimal(38,0),
          EST_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
          EST_SOURCE_ID  varchar(120),
          EST_SOURCE_SYSTEM_ID  varchar(120),
          EST_SOURCE_SYS_ORIGIN  varchar(120),
          EST_DESC  varchar(400))
        ;

        /* Populate DIFF Table - New/Updated Records */
        insert into gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_STATUS_ONB_PRODUCTRESULT_ADS_MAP_SCD_DIFF
        (
          tech_del_flg,
          tech_new_rec,
          tech_rid,
          EST_KEY,
          EST_SOURCE_ID,
          EST_SOURCE_SYSTEM_ID,
          EST_SOURCE_SYS_ORIGIN,
          EST_DESC
        )
        select   'N' as tech_del_flg, 
            case when trg.EST_SOURCE_ID is null then 'Y' else 'N' end as tech_new_rec,
            trg.rid as tech_rid,
           trg.EST_KEY, 
           src.EST_SOURCE_ID, 
           src.EST_SOURCE_SYSTEM_ID, 
           src.EST_SOURCE_SYS_ORIGIN, 
           src.EST_DESC
         from 
            (select /*+ full(xc) */      EST_SOURCE_ID, 
               EST_SOURCE_SYSTEM_ID, 
               EST_SOURCE_SYS_ORIGIN, 
               EST_DESC
               from gap_catalog.ads_etl_owner.XC_RDS_EVENT_STATUS_ONB_PRODUCTRESULT xc
            where (EST_SOURCE_SYSTEM_ID = 'RDS' and  EST_SOURCE_SYS_ORIGIN = 'RDS_ONBOARDINGPRODUCTRESULT')) src LEFT JOIN
            (select  est_key||'.'||est_valid_from as rid, t.* from gap_catalog.ads_owner.EVENT_STATUS t
              where EST_CURRENT_FLAG  = 'Y'
                and EST_VALID_TO  = DATE('3000-01-01')
               and (EST_SOURCE_SYSTEM_ID = 'RDS' and  EST_SOURCE_SYS_ORIGIN = 'RDS_ONBOARDINGPRODUCTRESULT')      ) trg
        ON trg.EST_SOURCE_ID = src.EST_SOURCE_ID
         and trg.EST_SOURCE_SYSTEM_ID = src.EST_SOURCE_SYSTEM_ID
         and trg.EST_SOURCE_SYS_ORIGIN = src.EST_SOURCE_SYS_ORIGIN
         and trg.EST_VALID_TO = DATE('3000-01-01') WHERE (
             decode( src.EST_DESC,trg.EST_DESC,1,0 ) = 0 or 
             trg.EST_SOURCE_ID is null or 
             trg.EST_DELETED_FLAG = 'Y'
           );

        /* Populate DIFF Table - Deleted Records */
        insert into gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_STATUS_ONB_PRODUCTRESULT_ADS_MAP_SCD_DIFF
        (
          tech_del_flg,
          tech_new_rec,
          tech_rid,
          EST_KEY,
          EST_SOURCE_ID,
          EST_SOURCE_SYSTEM_ID,
          EST_SOURCE_SYS_ORIGIN,
          EST_DESC
        )
        select   'Y' as tech_del_flg, 
         'N' as tech_new_rec, 
         trg.rid as tech_rid, 
           trg.EST_KEY, 
           trg.EST_SOURCE_ID, 
           trg.EST_SOURCE_SYSTEM_ID, 
           trg.EST_SOURCE_SYS_ORIGIN, 
           trg.EST_DESC
         from 
            (select  est_key||'.'||est_valid_from as rid, t.* from gap_catalog.ads_owner.EVENT_STATUS t
              where EST_CURRENT_FLAG  = 'Y' and EST_DELETED_FLAG  = 'N'
                and EST_VALID_TO  = DATE('3000-01-01')
               and (EST_SOURCE_SYSTEM_ID = 'RDS' and  EST_SOURCE_SYS_ORIGIN = 'RDS_ONBOARDINGPRODUCTRESULT')      ) trg LEFT JOIN
            (select /*+ full(xc) */      EST_SOURCE_ID, 
               EST_SOURCE_SYSTEM_ID, 
               EST_SOURCE_SYS_ORIGIN, 
               DATE('3000-01-01') as EST_VALID_TO
               from gap_catalog.ads_etl_owner.XC_RDS_EVENT_STATUS_ONB_PRODUCTRESULT xc
            where (EST_SOURCE_SYSTEM_ID = 'RDS' and  EST_SOURCE_SYS_ORIGIN = 'RDS_ONBOARDINGPRODUCTRESULT')) src
        ON trg.EST_SOURCE_ID = src.EST_SOURCE_ID
         and trg.EST_SOURCE_SYSTEM_ID = src.EST_SOURCE_SYSTEM_ID
         and trg.EST_SOURCE_SYS_ORIGIN = src.EST_SOURCE_SYS_ORIGIN
         and trg.EST_VALID_TO = src.EST_VALID_TO WHERE (src.EST_SOURCE_ID is null);

        /* Close Old Records in Target */
        update  gap_catalog.ads_owner.EVENT_STATUS set
               EST_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
               EST_UPDATE_PROCESS_KEY = CAST(p_process_key AS BIGINT),
               EST_CURRENT_FLAG = 'N', 
               EST_VALID_TO = DATEADD(DAY, -1, CAST(p_load_date AS DATE))
         where EST_CURRENT_FLAG = 'Y'
           and EST_VALID_TO = DATE('3000-01-01')
           and est_key||'.'||est_valid_from in (select tech_rid from gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_STATUS_ONB_PRODUCTRESULT_ADS_MAP_SCD_DIFF where tech_rid is not null);

        /* Insert Changed Records */
        insert  into gap_catalog.ads_owner.EVENT_STATUS 
         ( EST_KEY, 
           EST_SOURCE_ID, 
           EST_SOURCE_SYSTEM_ID, 
           EST_SOURCE_SYS_ORIGIN, 
           EST_DESC, 
           EST_VALID_FROM, 
           EST_VALID_TO, 
           EST_CURRENT_FLAG, 
           EST_DELETED_FLAG, 
           EST_INSERTED_DATETIME, 
           EST_INSERT_PROCESS_KEY, 
           EST_UPDATED_DATETIME, 
           EST_UPDATE_PROCESS_KEY)
        select EST_KEY, 
           EST_SOURCE_ID, 
           EST_SOURCE_SYSTEM_ID, 
           EST_SOURCE_SYS_ORIGIN, 
           EST_DESC, 
           CAST(p_load_date AS DATE) as EST_VALID_FROM, 
           DATE('3000-01-01') as EST_VALID_TO, 
           'Y' as EST_CURRENT_FLAG, 
           tech_del_flg as EST_DELETED_FLAG, 
           CURRENT_TIMESTAMP() as EST_INSERTED_DATETIME, 
           CAST(p_process_key AS BIGINT) as EST_INSERT_PROCESS_KEY, 
           CURRENT_TIMESTAMP() as EST_UPDATED_DATETIME, 
           CAST(p_process_key AS BIGINT) as EST_UPDATE_PROCESS_KEY
          from gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_STATUS_ONB_PRODUCTRESULT_ADS_MAP_SCD_DIFF
         where tech_new_rec = 'N';

        /* Insert New Records */
        insert  into gap_catalog.ads_owner.EVENT_STATUS 
         ( EST_KEY, 
           EST_SOURCE_ID, 
           EST_SOURCE_SYSTEM_ID, 
           EST_SOURCE_SYS_ORIGIN, 
           EST_DESC, 
           EST_VALID_FROM, 
           EST_VALID_TO, 
           EST_CURRENT_FLAG, 
           EST_DELETED_FLAG, 
           EST_INSERTED_DATETIME, 
           EST_INSERT_PROCESS_KEY, 
           EST_UPDATED_DATETIME, 
           EST_UPDATE_PROCESS_KEY)
        select 
           xxhash64(uuid()) AS EST_KEY,
           EST_SOURCE_ID, 
           EST_SOURCE_SYSTEM_ID, 
           EST_SOURCE_SYS_ORIGIN, 
           EST_DESC, 
           CAST(p_load_date AS DATE) as EST_VALID_FROM, 
           DATE('3000-01-01') as EST_VALID_TO, 
           'Y' as EST_CURRENT_FLAG, 
           tech_del_flg as EST_DELETED_FLAG, 
           CURRENT_TIMESTAMP() as EST_INSERTED_DATETIME, 
           CAST(p_process_key AS BIGINT) as EST_INSERT_PROCESS_KEY, 
           CURRENT_TIMESTAMP() as EST_UPDATED_DATETIME, 
           CAST(p_process_key AS BIGINT) as EST_UPDATE_PROCESS_KEY
          from gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_STATUS_ONB_PRODUCTRESULT_ADS_MAP_SCD_DIFF
         where tech_new_rec = 'Y';

        CALL gap_catalog.log.complete_performance_test(
            plog_id => my_log_id
        );

    END;
    
END;